We can replicate our SQLDatabaseChain with Runnables.

In [1]:
from langchain_core.prompts import ChatPromptTemplate

template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""
prompt = ChatPromptTemplate.from_template(template)

In [2]:
from langchain_community.utilities import SQLDatabase

We'll need the Chinook sample DB for this example. There's many places to download it from, e.g. https://database.guide/2-sample-databases-sqlite/

In [20]:
db = SQLDatabase.from_uri("sqlite:///./Chinook.db")

In [21]:
def get_schema(_):
    return db.get_table_info()

In [22]:
def run_query(query):
    return db.run(query)

In [23]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

sql_response = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | model.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

In [24]:
sql_response.invoke({"question": "How many employees are there?"})

'SELECT COUNT(*) FROM Employee'

In [25]:
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_template(template)

In [26]:
full_chain = (
    RunnablePassthrough.assign(query=sql_response).assign(
        schema=get_schema,
        response=lambda x: db.run(x["query"]),
    )
    | prompt_response
    | model
)

In [27]:
full_chain.invoke({"question": "How many employees are there?"})

AIMessage(content='There are 8 employees.', additional_kwargs={}, example=False)